In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/lib/spark/"
os.environ["PYSPARK_PYTHON"]= "/home/cloudera/anaconda2/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]= "/home/cloudera/anaconda2/bin/python"
os.environ["SPARK_YARN_USER_ENV"]= "/home/cloudera/anaconda2/bin/python"

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.1-src.zip'))
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

from pyspark import SparkContext, SparkConf

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:09:15)
SparkContext available as sc, HiveContext available as sqlContext.


##### The Kaggle Instatcart Market Basket analysis has data with more than 2 million rows in orders_product_prior and other datat set also has around a milloin rows that requires manipulation using partition and parallel processing.
##### For this reason In am using spark in jupyter and building my complete project in pycharm IDE. 
##### Created a virtual enviroment with spark libraries and anaconda python 2.7, initial settings are shown inthe first cell. 
##### Changed the 'dfs.blocksize' paramemter in the hdfs-default.xml to 64X1024X1024 bytes. 
##### This will increase parallisum in the hdfs processing for aggregating dataset in memory becomes faster.
##### for more information check this blog http://www.bigsynapse.com/spark-input-output.
##### Though I am using Pseudorandom distribution on cloudera quick start VM 5.8 it might not have any performance benifit as expected as machine is single but one cn try increasing the no of core and the executor memory.
##### default driver memory given is 5g in my VM , since in local mode executor is inside the driver so can't get memory more than that.##### underline properties to configure is spark  --num-executors <is one as installation is standalone> --executor-cores <depends on core of processor> --executor-memory <Memory available to the VM>

#### Order Schema 
##### order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_orde
##### read data with partition of three since data size of orders file is more than 110 mb we use 4 partition of size 3*32 mb
##### 32 mb is the local file system block size in whicj data is stored
##### 128 mb block size is the size of memory the name node referrences more 32 size blockes can be added to it.
##### It is for the namenode to decide in which 128 mb block refrence the 32 mb block can go

In [6]:
orders = sc.textFile("file:///home/cloudera/Desktop/kaggle/orders.csv", 4).filter(lambda x: 'order_id' not in x).map(lambda x : x.split(','))

In [90]:
orders.count()

3421083

#### Order_Product_Schema
##### order_id, Product_id, 
##### Here for we are taking the partition of 'order_products__prior' to be 18 as it is of more 550 mb 
##### similarly for 'order_products__train'  it is  default 0 partiton as the size is less than block size 32 mb

In [10]:
orders_prior = sc.textFile("file:///home/cloudera/Desktop/kaggle/order_products__prior.csv", 18).filter(lambda x: 'order_id' not in x).\
    map(lambda x : x.split(','))

orders_train = sc.textFile("file:///home/cloudera/Desktop/kaggle/order_products__train.csv").filter(lambda x: 'order_id' not in x).\
    map(lambda x : x.split(','))

In [21]:
orders_prior.take(1)

[[u'2', u'33120', u'1', u'1']]

##### Save all rdds created to hadoop file system in distributed fashion based on the no of the partions inthe rdd while loading 

In [23]:
orders.map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/orders")

In [24]:
orders_prior.map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/orders_prior")

In [25]:
orders_train.map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/orders_train")

In [28]:
## Define null prser to parse the blank strings 
def nullparse(x):
    if x =='':
        x=0
    else:
        float(x)
    return x

In [41]:
x= sc.textFile("/user/cloudera/kaggle/orders").map(lambda x: x.split(',')).map(lambda x : (int(x[0]),(x[1],x[2],x[3],x[4],x[5],str(nullparse(x[6])))))

In [38]:
y=sc.textFile("/user/cloudera/kaggle/orders_prior").map(lambda x: x.split(',')).map(lambda x : (int(x[0]), x[1]))

In [65]:
z= sc.textFile("/user/cloudera/kaggle/orders_train").map(lambda x: x.split(',')).map(lambda x : (int(x[0]), x[1]))

In [43]:
x.filter(lambda x : x[1][1]!="test").mapValues(lambda x : x[5]).reduceByKey(lambda x,y : x+y).\
    sortBy(lambda x : -x[1]).take(3)

##### Combine all products purchased for a perticular order id in one string sperated by space for order_product_prior data set.

In [45]:
y.groupByKey().mapValues(lambda x : ' '.join(x)).take(3)

[(3145734, u'45223 32740 46616 8230 6975 47668 3582'),
 (2621448, u'37718 38544 2748 47209 45504 28699 8006 41665 2078 35887'),
 (2097162, u'48988 37774 24852 19057')]

##### Save the intermediate result to the HDFS to reduce the time for further agregations

In [52]:
y.groupByKey().mapValues(lambda x : ' '.join(x)).map(lambda x : (str(x[0]),x[1])).map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/order_prior_inter")

In [53]:
order_prod_prio_inter = sc.textFile("/user/cloudera/kaggle/order_prior_inter").map(lambda x: x.split(','))

In [54]:
order_prod_prio_inter.take(2)

[[u'3145734', u'45223 32740 46616 8230 6975 47668 3582'],
 [u'2621448', u'37718 38544 2748 47209 45504 28699 8006 41665 2078 35887']]

In [57]:
order_prod_prio_inter.map(lambda x : (int(x[0]),x[1])).take(3)

[(3145734, u'45223 32740 46616 8230 6975 47668 3582'),
 (2621448, u'37718 38544 2748 47209 45504 28699 8006 41665 2078 35887'),
 (2097162, u'48988 37774 24852 19057')]

In [58]:
x.join(order_prod_prio_inter.map(lambda x : (int(x[0]),x[1]))).take(3)

[(2883584,
  ((u'78882', u'prior', u'1', u'3', u'20', '0'), u'43860 12249 34862')),
 (786434,
  ((u'29232', u'prior', u'16', u'0', u'12', '4.0'),
   u'29734 20847 4812 5491 31433 30192 45045 49628 38192 34134 26790 6891 7952 37658 28204 5194 22825 26346 32743 1158 18200 39977')),
 (1572868, ((u'14938', u'prior', u'7', u'0', u'14', '3.0'), u'21903'))]

In [61]:
x.join(order_prod_prio_inter.map(lambda x : (int(x[0]),x[1]))).mapValues(lambda x : x[0]+(x[1],)).map(lambda x : (str(x[0]), x[1])).\
map(lambda x : (x[0],) + x[1]).map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/order_product_prior_joined")

####order_product_prior_join Schema
##### order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order, all_purch_prod 


In [64]:
sc.textFile("/user/cloudera/kaggle/order_product_prior_joined").map(lambda x : x.split(',')).take(2)

[[u'2883584',
  u'78882',
  u'prior',
  u'1',
  u'3',
  u'20',
  u'0',
  u'43860 12249 34862'],
 [u'786434',
  u'29232',
  u'prior',
  u'16',
  u'0',
  u'12',
  u'4.0',
  u'29734 20847 4812 5491 31433 30192 45045 49628 38192 34134 26790 6891 7952 37658 28204 5194 22825 26346 32743 1158 18200 39977']]

In [75]:
z.groupByKey().mapValues(lambda x : ' '.join(x)).take(3)

[(1572864,
  u'19204 39698 48290 32000 40463 20682 13176 11526 33890 5025 14161 9387 13931 33198 43123 45351 5618'),
 (1, u'49302 11109 10246 49683 43633 13176 47209 22035'),
 (2359297, u'21781 4421 39009 36431 30233 5097 47209 28427 2513')]

##### Now lets create joined data set for order_product_train and the orders, this wont take much time as train data set is small and we can filter the orders data set for train only  

In [77]:
x.filter(lambda x : x[1][1]=='train').join(z.groupByKey().mapValues(lambda x : ' '.join(x))).mapValues(lambda x : x[0]+(x[1],)).map(lambda x : (str(x[0]), x[1])).\
map(lambda x : (x[0],) + x[1]).map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/order_product_train_joined")

#### order_product_train_join Schema
##### order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order, all_purch_prod

In [81]:
sc.textFile("/user/cloudera/kaggle/order_product_train_joined").take(3)

[u'1441800,23471,train,6,2,16,30.0,21713 24852 28204 3804 25544 8645 10916 29487 28634 17008',
 u'2621450,161225,train,25,4,13,30.0,46061',
 u'1703950,50594,train,4,4,12,30.0,19029 23594 8224 26882 18135 39922 38419 7042']

In [82]:
joined_train = sc.textFile("/user/cloudera/kaggle/order_product_train_joined")

In [83]:
joined_prior =  sc.textFile("/user/cloudera/kaggle/order_product_prior_joined")

In [89]:
joined_prior.map(lambda x : x.split(',')).map(lambda x : (int(x[0], int(x[1],int))))

0

In [93]:
subset_rdd_prior= sc.parallelize(joined_prior.take(10000))

In [106]:
subset_rdd_prior.take(2)

[u'2883584,78882,prior,1,3,20,0,43860 12249 34862',
 u'786434,29232,prior,16,0,12,4.0,29734 20847 4812 5491 31433 30192 45045 49628 38192 34134 26790 6891 7952 37658 28204 5194 22825 26346 32743 1158 18200 39977']

#### Combined Prior Order History Grouped by user
##### Now we have the combined data set based on the user from which we have to find insights 

In [113]:
joined_prior.map(lambda x : x.split(',')).map(lambda x : (int(x[1]), (x[0], x[3],x[4],x[6],x[7] ))).\
    combineByKey((lambda x : ((x[0],), (x[1],),(x[2],),(x[3],), (x[4],))), 
                 (lambda x,y: (x[0] + (y[0],),x[1] + (y[1],),x[2] + (y[2],),x[3] + (y[3],),x[4] + (y[4],))),
                 (lambda x,y : (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]))).take(10)

[(65538,
  ((u'2886687',
    u'2295459',
    u'2982586',
    u'3279300',
    u'1458602',
    u'2531850',
    u'1564378',
    u'1357315',
    u'848742',
    u'2174616',
    u'1726630',
    u'1847851',
    u'622275',
    u'2094779',
    u'2102018',
    u'1983262',
    u'2105407',
    u'1383433',
    u'1410274',
    u'499386',
    u'609586',
    u'1524281',
    u'740091',
    u'929490',
    u'941612',
    u'1063186',
    u'3158598',
    u'764140',
    u'1384101',
    u'476843',
    u'3115083',
    u'2509776',
    u'1878663',
    u'1153279',
    u'3041583',
    u'1888147',
    u'322077',
    u'2397932',
    u'2691303',
    u'3106399'),
   (u'15',
    u'28',
    u'1',
    u'24',
    u'27',
    u'30',
    u'35',
    u'8',
    u'36',
    u'13',
    u'31',
    u'5',
    u'32',
    u'34',
    u'21',
    u'37',
    u'18',
    u'16',
    u'40',
    u'17',
    u'26',
    u'9',
    u'7',
    u'38',
    u'23',
    u'14',
    u'33',
    u'29',
    u'6',
    u'22',
    u'11',
    u'4',
    u'25',
    